In [ ]:
!pip install casadi
from IPython.display import clear_output
import os
import time

import numpy as np
import casadi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

os.makedirs('images',exist_ok=True)
if os.name == 'nt':
    plt.rcParams['font.family'] = 'MS Gothic'
elif os.name == 'posix':
    !pip install japanize-matplotlib


clear_output()

# 定数

In [ ]:
g = 9.81 #重力加速度
m = 0.1125 #質点の質量
k_s = 2 #バネ定数
l = 0.1375 #バネの自然長

pos_0 = casadi.DM.zeros(3)
G = casadi.DM([0,0,g])

ns = 5 #バネの個数

nu = 3 #制御変数の次元
nx = 2*3*(ns-1)+3 #状態変数の次元

# コスト関数の重み
Q = casadi.diag([0,0,0]*(ns-1)+[12.5,12.5,12.5]+[125,125,125]*(ns-1))
Q_f = casadi.diag([0,0,0]*(ns-1)+[10,10,10]+[0,0,0]*(ns-1))
R = casadi.diag([0.5,0.5,0.5])

# 予測ホライズン等
T = 2
K = 20
dt = T/K

# 制約条件
x_lb = [-np.inf]*nx
x_ub = [np.inf]*nx
u_lb = [-5]*nu
u_ub = [5]*nu

x_ref = casadi.DM([0,0,0]*(ns-1)+[8,0,0]+[0,0,0]*(ns-1))

total = nx*(K+1) + nu*K

# 状態方程式・積分器・コスト関数

In [ ]:
def make_spring_force():
    pos_i = casadi.SX.sym("pos_i",3)
    pos_i1 = casadi.SX.sym("pos_i1",3)
    pos_rel = pos_i1 - pos_i
    norm = casadi.norm_2(pos_rel)
    force = k_s*(norm - l)*pos_rel/norm

    F = casadi.Function("F",[pos_i,pos_i1],[force],['pos_i','pos_i1'],['F'])
    return F

def make_f():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    F = make_spring_force()
    offset = 3*ns

    X_dot = []
    V_dot = []

    for k in range(ns-1):
        vel = states[offset+3*k:offset+3*(k+1)]
        X_dot.append(vel)

        pos = states[3*k:3*(k+1)]
        pos_n = states[3*(k+1):3*(k+2)]
        pos_p = states[3*(k-1):3*k] if k != 0 else pos_0
        vel_dot = (F(pos_i=pos,pos_i1=pos_n)["F"]-F(pos_i=pos_p,pos_i1=pos)["F"])/m - G
        V_dot.append(vel_dot)

    states_dot = casadi.vertcat(*X_dot,ctrls,*V_dot)

    f = casadi.Function("f",[states,ctrls],[states_dot],['x','u'],['x_dot'])
    return f

def make_RK4():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()

    r1 = f(x=states,u=ctrls)["x_dot"]
    r2 = f(x=states+dt*r1/2,u=ctrls)["x_dot"]
    r3 = f(x=states+dt*r2/2,u=ctrls)["x_dot"]
    r4 = f(x=states+dt*r3,u=ctrls)["x_dot"]

    states_next = states + dt*(r1+2*r2+2*r3+r4)/6

    RK4 = casadi.Function("RK4",[states,ctrls],[states_next],["x","u"],["x_next"])
    return RK4

def make_integrator():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()
    ode = f(x=states, u=ctrls)["x_dot"]

    dae = {"x":states,"p":ctrls,"ode":ode}

    I = casadi.integrator("I","cvodes",dae,0,dt)
    return I

def compute_stage_cost(x,u):
    x_diff = x - x_ref
    cost = (casadi.dot(Q@x_diff,x_diff) + casadi.dot(R@u,u)) / 2
    return cost

def compute_terminal_cost(x):
    x_diff = x - x_ref
    cost = casadi.dot(Q_f@x_diff,x_diff) / 2
    return cost

# 最適化問題

In [ ]:
def make_nlps():
    RK4 = make_RK4()

    U = [casadi.SX.sym(f"u_{k}",nu) for k in range(K)]
    X = [casadi.SX.sym(f"x_{k}",nx) for k in range(K+1)]
    G = []

    J = 0

    for k in range(K):
        J += compute_stage_cost(X[k],U[k])*dt
        eq = X[k+1] - RK4(x=X[k],u=U[k])["x_next"]
        G.append(eq)
    J += compute_terminal_cost(X[-1])

    nlp = {"x":casadi.vertcat(*X,*U),"f":J,"g":casadi.vertcat(*G)}
    option = {'print_time':False,"ipopt":{'print_level':0}}
    S_ipopt = casadi.nlpsol("S","ipopt",nlp,option)
    
    option = {'print_time':False,
              "print_iteration":False,
              "print_header":False,
              "qpsol_options":{"print_info":False,"print_iter":False,"print_header":False}
              }
    S_qrsqp = casadi.nlpsol("S","qrsqp",nlp,option)
    return S_ipopt, S_qrsqp

# 最適な制御入力

In [ ]:
def compute_optimal_control(S,x_init,x0):
    x_init = x_init.full().ravel().tolist()
    
    lbx = x_init + x_lb*K + u_lb*K
    ubx = x_init + x_ub*K + u_ub*K
    lbg = [0]*nx*K
    ubg = [0]*nx*K

    res = S(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0)
    
    offset = nx*(K+1)
    x0 = res["x"]
    u_opt = x0[offset:offset+nu]
    return u_opt, x0

# 初期値の作成

In [ ]:
def make_x_init():
    x_init = np.zeros((2*ns-1,3))
    x_init[:ns,0] = np.linspace(0,8,ns+1)[1:]
    x_init[ns+1,1] = 8
    x_init = x_init.ravel()
    x_init = casadi.DM(x_init)
    return x_init

def make_x0(x_init):
    I = make_integrator()
    x0 = [x_init]
    x_tmp = x_init
    for i in range(K):
        x_tmp = I(x0=x_tmp)["xf"]
        x0.append(x_tmp)
    u0 = casadi.DM.zeros(nu*K)
    x0 = casadi.vertcat(*x0,u0)
    return x0

# 自由運動

In [ ]:
S_ipopt, S_qrsqp = make_nlps()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)
I = make_integrator()

x_init = make_x_init()
x0 = make_x0(x_init)

# Times = []
X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    u_opt = casadi.DM.zeros(nu)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

# 結果の可視化

In [ ]:
X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(3):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("State")

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Control")

plt.savefig("images/chap7_nlp_free.png")
plt.show()

In [ ]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(111,projection="3d")
# frames = np.arange(0,t_eval.size)
# fps = 1 / dt

# def update_figure(i):
#     x_lim_min = -1
#     x_lim_max = 8
#     y_lim_min = -1
#     y_lim_max = 1
#     z_lim_min = -3
#     z_lim_max = 1
#     u_scale = 3

#     ax.cla()
#     ax.set_xlim(x_lim_min, x_lim_max)
#     ax.set_ylim(y_lim_min, y_lim_max)
#     ax.set_zlim(z_lim_min, z_lim_max)
#     ax.view_init(elev=5,azim=-75)

#     points = X[i,:3*ns]
#     points = np.concatenate([np.zeros(3),points])
#     points = points.reshape(-1,3)
#     vel = U[i]

#     ax.scatter(*points.T,color="black", s=2)
#     ax.plot(*points.T, color='black', lw=2)
#     ax.quiver(*points[-1],*vel/u_scale, length=1.0,normalize=False)

#     left = np.zeros(3)
#     right = x_ref.full().ravel()[3*(ns-1):3*ns]
#     ax.scatter(*left,color="green", s=10)
#     ax.scatter(*right,color="blue", s=10)

# ani = FuncAnimation(fig, update_figure, frames=frames)
# ani.save("images/chap7_chain_of_mass.gif",writer="pillow",fps=fps)

# アニメーションの作成

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111,projection="3d")
frames = np.arange(0,t_eval.size)
fps = 1 / dt

def update_figure(i):
    x_lim_min = -1
    x_lim_max = 8
    y_lim_min = -1
    y_lim_max = 1
    z_lim_min = -3
    z_lim_max = 1
    u_scale = 3

    ax.cla()
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.set_ylim(y_lim_min, y_lim_max)
    ax.set_zlim(z_lim_min, z_lim_max)
    ax.view_init(elev=5,azim=-75)

    points = X[i,:3*ns]
    points = np.concatenate([np.zeros(3),points])
    points = points.reshape(-1,3)
    vel = U[i]

    ax.scatter(*points.T,color="black", s=2)
    ax.plot(*points.T, color='black', lw=2)
    ax.quiver(*points[-1],*vel/u_scale, length=1.0,normalize=False)

    left = np.zeros(3)
    right = x_ref.full().ravel()[3*(ns-1):3*ns]
    ax.scatter(*left,color="green", s=10)
    ax.scatter(*right,color="blue", s=10)

ani = FuncAnimation(fig, update_figure, frames=frames)
ani.save("images/chap7_chain_of_mass_free.gif",writer="pillow",fps=fps)

# IPOPTの場合

In [ ]:
S_ipopt, S_qrsqp = make_nlps()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)
I = make_integrator()

x_init = make_x_init()
x0 = make_x0(x_init)

Times_ipopt = []
X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    start_time = time.perf_counter()
    u_opt,x0 = compute_optimal_control(S_ipopt,x_current,x0)
    Times_ipopt.append(time.perf_counter() - start_time)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

# 結果の可視化

In [ ]:
X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(3):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("State")

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Control")

plt.savefig("images/chap7_nlp_ipopt.png")
plt.show()

# 計算時間

In [ ]:
Times_ipopt = np.array(Times_ipopt)
print(f"[*] solver: ipopt, elapsed time: {Times_ipopt.mean():1f}s")
plt.plot(Times_ipopt)
plt.xlabel("Time")
plt.ylabel("Elapsed Time")

plt.savefig("images/chap7_nlp_ipopt_time.png")
plt.show()

In [ ]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(111,projection="3d")
# frames = np.arange(0,t_eval.size)
# fps = 1 / dt

# def update_figure(i):
#     x_lim_min = -1
#     x_lim_max = 8
#     y_lim_min = -1
#     y_lim_max = 1
#     z_lim_min = -3
#     z_lim_max = 1
#     u_scale = 3

#     ax.cla()
#     ax.set_xlim(x_lim_min, x_lim_max)
#     ax.set_ylim(y_lim_min, y_lim_max)
#     ax.set_zlim(z_lim_min, z_lim_max)
#     ax.view_init(elev=5,azim=-75)

#     points = X[i,:3*ns]
#     points = np.concatenate([np.zeros(3),points])
#     points = points.reshape(-1,3)
#     vel = U[i]

#     ax.scatter(*points.T,color="black", s=2)
#     ax.plot(*points.T, color='black', lw=2)
#     ax.quiver(*points[-1],*vel/u_scale, length=1.0,normalize=False)

#     left = np.zeros(3)
#     right = x_ref.full().ravel()[3*(ns-1):3*ns]
#     ax.scatter(*left,color="green", s=10)
#     ax.scatter(*right,color="blue", s=10)

# ani = FuncAnimation(fig, update_figure, frames=frames)
# ani.save("images/chap7_chain_of_mass.gif",writer="pillow",fps=fps)

# アニメーションの作成

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111,projection="3d")
frames = np.arange(0,t_eval.size)
fps = 1 / dt

def update_figure(i):
    x_lim_min = -1
    x_lim_max = 8
    y_lim_min = -1
    y_lim_max = 1
    z_lim_min = -3
    z_lim_max = 1
    u_scale = 3

    ax.cla()
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.set_ylim(y_lim_min, y_lim_max)
    ax.set_zlim(z_lim_min, z_lim_max)
    ax.view_init(elev=5,azim=-75)

    points = X[i,:3*ns]
    points = np.concatenate([np.zeros(3),points])
    points = points.reshape(-1,3)
    vel = U[i]

    ax.scatter(*points.T,color="black", s=2)
    ax.plot(*points.T, color='black', lw=2)
    ax.quiver(*points[-1],*vel/u_scale, length=1.0,normalize=False)

    left = np.zeros(3)
    right = x_ref.full().ravel()[3*(ns-1):3*ns]
    ax.scatter(*left,color="green", s=10)
    ax.scatter(*right,color="blue", s=10)

ani = FuncAnimation(fig, update_figure, frames=frames)
ani.save("images/chap7_chain_of_mass.gif",writer="pillow",fps=fps)

# qrsqpの場合

In [ ]:
S_ipopt, S_qrsqp = make_nlps()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)
I = make_integrator()

x_init = make_x_init()
x0 = make_x0(x_init)

Times_qrsqp = []
X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    start_time = time.perf_counter()
    u_opt,x0 = compute_optimal_control(S_qrsqp,x_current,x0)
    Times_qrsqp.append(time.perf_counter() - start_time)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

# 結果の可視化

In [ ]:
X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(3):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("State")

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Control")

plt.savefig("images/chap7_nlp_qrsqp.png")
plt.show()

# 計算時間

In [ ]:
Times_qrsqp = np.array(Times_qrsqp)
print(f"[*] solver: qrsqp, elapsed time: {Times_qrsqp.mean():1f}s")
plt.plot(Times_qrsqp)
plt.xlabel("Time")
plt.ylabel("Elapsed Time")

plt.savefig("images/chap7_nlp_qrsqp_time.png")
plt.show()

# 計算時間の比較

In [ ]:
plt.plot(Times_ipopt,label="IPOPT")
plt.plot(Times_qrsqp,label="qrsqp")
plt.xlabel("Time")
plt.ylabel("Elapsed Time")
plt.legend()

plt.savefig("images/chap7_nlp_time_comparison.png")
plt.show()

# IPOPT (初期値がランダム)

In [ ]:
S_ipopt, S_qrsqp = make_nlps()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)
I = make_integrator()

x_init = make_x_init()
x0 = casadi.DM(0.05*np.random.normal(size=total))

Times = []
X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    start_time = time.perf_counter()
    u_opt,x0 = compute_optimal_control(S_ipopt,x_current,x0)
    Times.append(time.perf_counter() - start_time)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

# 結果の可視化

In [ ]:
X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(3):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("State")

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Control")

plt.savefig("images/chap7_nlp_ipopt_without_init.png")
plt.show()

# 計算時間

In [ ]:
Times = np.array(Times)
print(f"[*] solver: ipopt, elapsed time: {Times.mean():1f}s")
plt.plot(Times)
plt.xlabel("Time")
plt.ylabel("Elapsed Time")

plt.savefig("images/chap7_nlp_ipopt_time_without_init.png")
plt.show()

# qrsqp（初期値がランダム）

In [ ]:
S_ipopt, S_qrsqp = make_nlps()

t_span = [0,10]
t_eval = np.arange(*t_span,dt)
I = make_integrator()

x_init = make_x_init()
x0 = casadi.DM(0.05*np.random.normal(size=total))

Times = []
X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    start_time = time.perf_counter()
    u_opt,x0 = compute_optimal_control(S_qrsqp,x_current,x0)
    Times.append(time.perf_counter() - start_time)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(3):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()

plt.show()